# DATA READING

### Data Reading JSON

In [ ]:
df_json = spark.read.format('json').option('inferSchema',True)\
                    .option('header',True)\
                    .option('multiLine',False)\
                    .load('/FileStore/tables/drivers.json')

In [ ]:
df_json.display()

### Data Reading Utils

In [ ]:
dbutils.fs.ls('/FileStore/tables/')

In [ ]:
df = spark.read.format('csv').option('inferSchema',True).option('header',True).load('/FileStore/tables/BigMart_Sales.csv')

In [ ]:
df.display()

### Schema Definition

In [ ]:
df.printSchema()

### DDL SCHEMA

In [ ]:
my_ddl_schema = '''
                    Item_Identifier STRING,
                    Item_Weight STRING,
                    Item_Fat_Content STRING,
                    Item_Visibility DOUBLE,
                    Item_Type STRING,
                    Item_MRP DOUBLE,
                    Outlet_Identifier STRING,
                    Outlet_Establishment_Year INT,
                    Outlet_Size STRING,
                    Outlet_Location_Type STRING,
                    Outlet_Type STRING,
                    Item_Outlet_Sales DOUBLE

                '''

In [ ]:
df = spark.read.format('csv')\
            .schema(my_ddl_schema)\
            .option('header',True)\
            .load('/FileStore/tables/BigMart_Sales.csv')

In [ ]:
df.display()

In [ ]:
df.printSchema()

### StructType() Schema

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

my_strct_schema = StructType([
                                StructField('Item_Identifier',StringType(),True),
                                StructField('Item_Weight',StringType(),True),
                                StructField('Item_Fat_Content',StringType(),True),
                                StructField('Item_Visibility',StringType(),True),
                                StructField('Item_MRP',StringType(),True),
                                StructField('Outlet_Identifier',StringType(),True),
                                StructField('Outlet_Establishment_Year',StringType(),True),
                                StructField('Outlet_Size',StringType(),True),
                                StructField('Outlet_Location_Type',StringType(),True),
                                StructField('Outlet_Type',StringType(),True),
                                StructField('Item_Outlet_Sales',StringType(),True)

])

df = spark.read.format('csv')\
            .schema(my_strct_schema)\
            .option('header',True)\
            .load('/FileStore/tables/BigMart_Sales.csv')

In [ ]:
df.printSchema()

# TRANSFORMATIONS

### SELECT

In [ ]:
df.display()

In [ ]:
df.select(col('Item_Identifier'),col('Item_Weight'),col('Item_Fat_Content')).display()

### ALIAS

In [ ]:
df.select(col('Item_Identifier').alias('Item_ID')).display()

In [ ]:
df.display()

### FILTER

#### Scenario - 1

In [ ]:
df.filter(col('Item_Fat_Content')=='Regular').display()

#### Scenario - 2

In [ ]:
df.filter((col('Item_Type') == 'Soft Drinks') & (col('Item_Weight')<10)).display()

#### Scenario - 3

In [ ]:
df.filter((col('Outlet_Size').isNull()) & (col('Outlet_Location_Type').isin('Tier 1','Tier 2'))).display()

### withColumnRenamed

In [ ]:
df.withColumnRenamed('Item_Weight','Item_Wt').display()

### withColumn

#### Scenario - 1

In [ ]:
df = df.withColumn('flag',lit("new"))

In [2]:
df.display()

NameError: name 'df' is not defined

In [ ]:
df.withColumn('multiply',col('Item_Weight')*col('Item_MRP')).display()

#### Scenario - 2

In [3]:
df = df.withColumn('Item_Fat_Content',regexp_replace(col('Item_Fat_Content'),"Regular","Reg"))\
    .withColumn('Item_Fat_Content',regexp_replace(col('Item_Fat_Content'),"Low Fat","Lf"))

df.display()

NameError: name 'df' is not defined

### Type Casting

In [ ]:
df = df.withColumn('Item_Weight', col('Item_Weight').cast(StringType()))

In [ ]:
df.printSchema()

### sort

#### Scenario - 1

In [ ]:
df.sort(col('Item_Weight').desc()).display()

#### Scenario - 2

In [ ]:
df.sort(col('Item_Visibility').asc()).display()

#### Scenario - 3

In [ ]:
df.sort(['Item_Weight','Item_Visibility'],ascending = [0,0]).display()

#### Scenario - 4

In [ ]:
df.sort(['Item_weight','Item_Visibility'], ascending = [0,1]).display()

### Limit

In [ ]:
df.limit(10).display()

### DROP

#### Scenario-1

In [4]:
df.drop('Item_Visibility').display()

NameError: name 'df' is not defined

#### Scenario-2

In [ ]:
df.drop('Item_Visibility','Item_Type').display()

### DRop_Duplicates

In [ ]:
df.dropDuplicates().display()

#### Scenario - 2

In [ ]:
df.drop_duplicates(subset=['Item_Type']).display()

In [ ]:
df.distinct().display()

### UNION and UNION BY NAME

####Preaparing Dataframes

In [ ]:
data1 = [('1','kad'),
        ('2','sid')]
schema1 = 'id STRING, name STRING'

df1 = spark.createDataFrame(data1,schema1)

data2 = [('3','rahul'),
        ('4','jas')]
schema2 = 'id STRING, name STRING'

df2 = spark.createDataFrame(data2,schema2)



In [ ]:
df1.display()

In [ ]:
df2.display()

### Union

In [ ]:
df1.union(df2).display()

In [ ]:
data1 = [('kad','1',),
        ('sid','2',)]
schema1 = 'name STRING, id STRING'

df1 = spark.createDataFrame(data1,schema1)

df1.display()

In [ ]:
df1.union(df2).display()

### Union by Name

In [ ]:
df1.unionByName(df2).display()

### String Functions

#### Initcap()

In [ ]:
df.select(upper('Item_Type').alias('upper_Item_Type')).display()

### Date Functions

#### Current_Date

In [ ]:
df = df.withColumn('curr_date',current_date())

df.display()

#### Date_Add()

In [ ]:
df = df.withColumn('week_after',date_add('curr_date',7))

df.display()

#### Date_Sub()

In [ ]:
df.withColumn('week_before',date_sub('curr_date',7)).display()

In [ ]:
df = df.withColumn('week_before',date_add('curr_date',-7))

df.display()

### DateDIFF

In [ ]:
df = df.withColumn('datediff',datediff('week_after','curr_date'))

df.display()

### Date_Format()

In [ ]:
df = df.withColumn('week_before',date_format('week_before','dd-MM-yyyy'))

df.display()